In [27]:
try:
    from splinter import Browser
    from bs4 import BeautifulSoup as soup
    import pprint
    import pandas as pd
    import configparser
    import re
    print("Imported successfully")
except ImportError as e: # If failed to import, return notice
    print("Import faliure:", e)
    
config = configparser.ConfigParser()
config.read('config.ini')

username = config['credentials']['username']
password = config['credentials']['password']
csv_save_path = config.get('Paths', 'clean_csv')
json_path = config.get('JSON', 'location')
my_email = config.get('credentials', 'my_email')
folder_id = config.get('Paths', 'drive_folder')

Imported successfully


In [ ]:
# Initialize splinter/browser
browser = Browser('chrome')

# Visit hymnal
url = 'https://hymnal.pcusastore.com/Hymn/HymnSearch'
browser.visit(url)

browser.fill('UserName', (username))
browser.fill('Password', (password))
browser.find_by_css('.linkbutton-login').first.click() #click the login button

browser.find_by_css('ul.nav a[href="/Hymn/HymnSearch"]').first.click()
# Find the 'Hymn #' link by XPath and click it
hymn_sort_link_xpath = "//a[contains(@class, 'k-link') and contains(text(), 'Hymn #')]"
browser.find_by_xpath(hymn_sort_link_xpath).first.click()

# Click on the first hymn
browser.find_by_css('a[href="/Hymn/Index/1"]').click()

# After logging in, parse the first hymn's HTML with Beautiful Soup
first_hymn_html = browser.html
html_soup = soup(first_hymn_html, 'html.parser')

main_content = html_soup.find('div', class_='mainContent')

# Extract the text nodes (ignoring any nested tags like <h2>, <h3> etc.)
text_lines = [text for text in main_content.stripped_strings]

In [ ]:
def split_into_pairs(lyrics):
    # Split the lyrics into lines using a regular expression
    lines = re.split(r'\n+', lyrics)
    # Remove empty lines and verse numbers
    lines = [re.sub(r'^\d+\s*', '', line) for line in lines if line.strip()]
    # Group lines into pairs
    pairs = ['\n'.join(lines[i:i + 2]) for i in range(0, len(lines), 2)]
    return pairs

# Initialize an empty list to store hymn data
hymns_data = []

for hymn_number in range(2, 854):
# for hymn_number in range(2, 5):  # Reduced range for testing
    url = f"https://hymnal.pcusastore.com/Hymn/Index/{hymn_number}"
    browser.visit(url)
    html = browser.html
    html_soup = soup(html, 'html.parser')

    hymn_title_element = html_soup.find('h2', class_='hymnTitle')
    hymn_title = hymn_title_element.get_text(strip=True) if hymn_title_element else 'Title not found'

    main_content = html_soup.find('div', class_='mainContent')
    if main_content:
        # Replace <br> tags with newlines
        for br in main_content.find_all("br"):
            br.replace_with("\n")
        cleaned_lyrics = main_content.get_text(separator='\n', strip=True)

        # Extract only the lyrics part
        start_idx = cleaned_lyrics.find('Lyrics') + len('Lyrics')
        end_idx = cleaned_lyrics.find('Informational Notes', start_idx)
        lyrics_only = cleaned_lyrics[start_idx:end_idx].strip()

        lyric_pairs = split_into_pairs(lyrics_only)

        for pair in lyric_pairs:
            hymns_data.append({'HymnNumber': hymn_number, 'Title': hymn_title, 'LyricPair': pair})

# Create DataFrame
df_hymns = pd.DataFrame(hymns_data)

In [ ]:
df_hymns.head()

In [ ]:
df_hymns.to_csv(csv_save_path, index=False)

In [ ]:
# Close the brower's connection
browser.quit()

## Let's import them into PowerPoint now...

In [23]:
try:
    import time
    import random
    import uuid
    import hashlib
    import logging
    from google.oauth2 import service_account
    from googleapiclient.discovery import build
    from googleapiclient.errors import HttpError
    import random
    import time
    import logging
    print('Import sucessful')
except ImportError as e:
    print('Import Failed', e)

Import sucessful


In [34]:
# Load the cleaned data
df_hymns = pd.read_csv(csv_save_path)

# Authenticate and create service objects
SCOPES = ['https://www.googleapis.com/auth/presentations', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = json_path

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

slides_service = build('slides', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

def move_file_to_folder(drive_service, file_id, folder_id):
    # Retrieve the existing parents to remove
    file = drive_service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))
    # Move the file to the new folder
    drive_service.files().update(
        fileId=file_id,
        addParents=folder_id,
        removeParents=previous_parents,
        fields='id, parents'
    ).execute()

def create_folder_if_not_exists(drive_service, folder_name):
    # Search for the folder
    response = drive_service.files().list(
        q=f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed=false",
        spaces='drive',
        fields='files(id, name)').execute()
    
    folders = response.get('files', [])
    
    # If folder exists, return its ID
    if folders:
        return folders[0]['id']
    
    # Folder does not exist, so create it
    folder_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

# Set up logging for debug
logging.basicConfig(filename='hymn_slides_creation.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to handle exponential backoff
def exponential_backoff_with_logging(retry_number, max_retries):
    max_wait_time_seconds = 600  # Up to 10 minutes
    wait_time_seconds = min((2 ** retry_number) + random.randint(0, 1000), max_wait_time_seconds * 1000) / 1000.0
    logging.info(f'Exponential backoff: waiting for {wait_time_seconds} seconds before retrying ({retry_number + 1}/{max_retries})...')
    time.sleep(wait_time_seconds)
    
def create_presentation(service, title):
    try:
        presentation_body = {'title': title}
        presentation = service.presentations().create(body=presentation_body).execute()
        return presentation.get('presentationId')
    except HttpError as error:
        logging.error(f'HttpError while creating presentation: {error}')
        raise
    except Exception as e:
        logging.error(f'Unexpected error while creating presentation: {e}')
        raise

def add_slide_with_text_box(service, presentation_id, text):
    # Create a new slide request without specifying an object ID
    new_slide_request = {'createSlide': {}}

    # Execute the request to create a new slide
    slide_response = service.presentations().batchUpdate(
        presentationId=presentation_id,
        body={'requests': [new_slide_request]}
    ).execute()

    # Retrieve the ID of the newly created slide
    slide_id = slide_response.get('replies')[0].get('createSlide').get('objectId')

    # Unique ID for the textbox
    textbox_id = 'textbox_' + str(uuid.uuid4())

    # Convert inches to EMUs for dimensions and positions
    inch_to_emu = 914400
    width = 7 * inch_to_emu  # 7 inches width (8.5 - 1.5)
    height = (1.2 + 2/8) * inch_to_emu  # Height to stop at 1.2 inches from the top (1 + 2/8 inches from the top)
    x_pos = 1.5 * inch_to_emu  # X position 1.5 inches from the left
    y_pos = 2/8 * inch_to_emu  # Y position 2/8 inches from the top

    # Create textbox request
    textbox_request = {
        'createShape': {
            'objectId': textbox_id,
            'shapeType': 'TEXT_BOX',
            'elementProperties': {
                'pageObjectId': slide_id,
                'size': {
                    'height': {'magnitude': height, 'unit': 'EMU'},
                    'width': {'magnitude': width, 'unit': 'EMU'}
                },
                'transform': {
                    'scaleX': 1,
                    'scaleY': 1,
                    'translateX': x_pos,
                    'translateY': y_pos,
                    'unit': 'EMU'
                }
            }
        }
    }

    # Add padding lines for vertical centering
    vertical_padding = '\n'  # Adjust the number of newline characters based on your needs
    padded_text = vertical_padding + text

    # Insert text into the textbox
    insert_text_request = {
        'insertText': {
            'objectId': textbox_id,
            'text': padded_text
        }
    }

    # Set the text style
    text_style_request = {
        'updateTextStyle': {
            'objectId': textbox_id,
            'style': {
                'fontFamily': 'Comfortaa',
                'fontSize': {'magnitude': 18, 'unit': 'PT'},
                'foregroundColor': {'opaqueColor': {'rgbColor': {'red': 0, 'green': 0, 'blue': 0}}},
                'bold': False
            },
            'fields': 'fontFamily,fontSize,foregroundColor,bold'
        }
    }

    # Set the paragraph alignment
    paragraph_style_request = {
        'updateParagraphStyle': {
            'objectId': textbox_id,
            'style': {
                'alignment': 'CENTER'
            },
            'fields': 'alignment',
            'textRange': {'type': 'ALL'}
        }
    }

    # Update background color request for the textbox
    background_color_request = {
        'updateShapeProperties': {
            'objectId': textbox_id,
            'shapeProperties': {
                'shapeBackgroundFill': {
                    'solidFill': {
                        'color': {
                            'rgbColor': {
                                'red': 183 / 255, 
                                'green': 183 / 255, 
                                'blue': 183 / 255
                            }
                        },
                        'alpha': 1.0  # Full opacity
                    }
                },
                'outline': {
                    'outlineFill': {
                        'solidFill': {
                            'color': {
                                'rgbColor': {'red': 0, 'green': 0, 'blue': 0}  # Black color for border
                            }
                        }
                    },
                    'weight': {'magnitude': 12700, 'unit': 'EMU'}  # Border width (1 px)
                }
            },
            'fields': 'shapeBackgroundFill,outline'
        }
    }

    # Execute the requests to add the text box, insert text, and format it
    requests = [textbox_request, insert_text_request, text_style_request, paragraph_style_request, background_color_request]
    service.presentations().batchUpdate(
        presentationId=presentation_id, 
        body={'requests': requests}
    ).execute()

# Initialize a list to store presentation IDs
presentation_ids = []

# Initialize the 'Hymn Lyrics' folder
folder_id = create_folder_if_not_exists(drive_service, 'Hymn Lyrics')

# Define a small delay (e.g., 30 seconds) between processing each hymn
processing_delay = 30

# Filter the DataFrame for a subset of hymns (e.g., 2 to 10)
# subset_df_hymns = df_hymns[df_hymns['HymnNumber'].between(2, 10)] # For debugging

# Iterate through each hymn and create a new presentation with slides
for index, hymn in df_hymns.drop_duplicates(subset='HymnNumber').iterrows():
# for index, hymn in subset_df_hymns.drop_duplicates(subset='HymnNumber').iterrows(): # Small subset for debugging
    retry = 0
    max_retries = 10  # Set the maximum number of retries
    while retry < max_retries:
        try:
            # Create a new presentation
            presentation_body = {'title': hymn['Title']}
            presentation = slides_service.presentations().create(body=presentation_body).execute()
            presentation_id = presentation.get('presentationId')
            
            # Move the created presentation to the 'Hymn Lyrics' folder
            move_file_to_folder(drive_service, presentation_id, folder_id)
            
            # Append the presentation ID to the list
            presentation_ids.append(presentation_id)

            # Filter the specific hymn lyrics
            hymn_lyrics = df_hymns[df_hymns['HymnNumber'] == hymn['HymnNumber']]

            # Add each lyric pair as a separate slide in the same presentation
            for _, lyric_row in hymn_lyrics.iterrows():
                add_slide_with_text_box(slides_service, presentation_id, lyric_row['LyricPair'])

            # Print a message for creating the presentation
            print(f"Created presentation for hymn number {hymn['HymnNumber']} - '{hymn['Title']}'")
            
            # Wait before processing the next hymn
            time.sleep(processing_delay)
            break  # Break the outer loop if successful
        except HttpError as error:
            if error.resp.status in [429, 500, 503]:  # Rate limit or server errors
                logging.warning(f'HttpError {error.resp.status}: {error}. Retrying...')
                exponential_backoff_with_logging(retry, max_retries)
                retry += 1
            else:
                logging.error(f'Non-retriable HttpError: {error}')
                raise
        except Exception as e:
            logging.error(f'Unexpected error: {e}')
            break  # Stop the loop in case of non-HTTP errors


Created presentation for hymn number 2 - 'Come, Thou Almighty King'
Created presentation for hymn number 3 - 'Womb of Life and Source of Being'
Created presentation for hymn number 4 - 'Holy God, We Praise Your Name'
Created presentation for hymn number 5 - 'God the Sculptor of the Mountains'
Created presentation for hymn number 6 - 'I Bind unto Myself Today'
Created presentation for hymn number 7 - 'Mothering God, You Gave Me Birth'
Created presentation for hymn number 8 - 'Eternal Father, Strong to Save'
Created presentation for hymn number 9 - 'The Play of the Godhead'
Created presentation for hymn number 10 - 'Sing Glory to the Name of God'
Created presentation for hymn number 11 - 'Source and Sovereign, Rock and Cloud'
Created presentation for hymn number 12 - 'Immortal, Invisible, God Only Wise'
Created presentation for hymn number 13 - 'The Mighty God, with Power Speaks'
Created presentation for hymn number 14 - 'For the Beauty of the Earth'
Created presentation for hymn number 

Created presentation for hymn number 117 - 'While Shepherds Watched Their Flocks'
Created presentation for hymn number 118 - 'While Shepherds Watched Their Flocks'
Created presentation for hymn number 119 - 'Hark! The Herald Angels Sing'
Created presentation for hymn number 120 - 'Where Shepherds Lately Knelt'
Created presentation for hymn number 121 - 'O Little Town of Bethlehem'
Created presentation for hymn number 122 - 'Silent Night, Holy Night!'
Created presentation for hymn number 123 - 'It Came Upon the Midnight Clear'
Created presentation for hymn number 124 - 'Still, Still, Still'
Created presentation for hymn number 125 - 'Before the Marvel of This Night'
Created presentation for hymn number 126 - 'Jesus, Jesus, O What a Wonderful Child'
Created presentation for hymn number 127 - 'Hark! The Herald Angels Sing'
Created presentation for hymn number 128 - 'Infant Holy, Infant Lowly'
Created presentation for hymn number 129 - 'Lo, How a Rose E'er Blooming'
Created presentation fo

Created presentation for hymn number 232 - 'Jesus Christ Is Risen Today'
Created presentation for hymn number 233 - 'The Day of Resurrection!'
Created presentation for hymn number 234 - 'Come, You Faithful, Raise the Strain'
Created presentation for hymn number 235 - 'O Sons and Daughters, Let Us Sing'
Created presentation for hymn number 236 - 'The Strife Is O'er'
Created presentation for hymn number 237 - 'Christ Jesus Lay in Death's Strong Bands'
Created presentation for hymn number 238 - 'Thine Is the Glory'
Created presentation for hymn number 239 - 'Good Christians All, Rejoice and Sing!'
Created presentation for hymn number 240 - 'Alleluia, Alleluia! Give Thanks'
Created presentation for hymn number 241 - 'Woman, Weeping in the Garden'
Created presentation for hymn number 242 - 'Day of Delight and Beauty Unbounded'
Created presentation for hymn number 243 - 'Be Not Afraid'
Created presentation for hymn number 245 - 'Christ the Lord Is Risen Today!'
Created presentation for hymn 

Created presentation for hymn number 344 - 'Where Armies Scourge the Countryside'
Created presentation for hymn number 345 - 'In an Age of Twisted Values'
Created presentation for hymn number 346 - 'For the Healing of the Nations'
Created presentation for hymn number 347 - 'Let All Mortal Flesh Keep Silence'
Created presentation for hymn number 348 - 'Lo, He Comes with Clouds Descending'
Created presentation for hymn number 349 - '"Sleepers, Wake!" A Voice Astounds Us'
Created presentation for hymn number 350 - 'Keep Your Lamps Trimmed and Burning'
Created presentation for hymn number 351 - 'All Who Love and Serve Your City'
Created presentation for hymn number 352 - 'My Lord! What a Morning'
Created presentation for hymn number 353 - 'My Hope Is Built on Nothing Less'
Created presentation for hymn number 354 - 'Mine Eyes Have Seen the Glory'
Created presentation for hymn number 355 - 'O Hear Our Cry, O Lord'
Created presentation for hymn number 356 - 'Sing Praise to God, Whose Mighty 

Created presentation for hymn number 461 - 'As Dew Falls Gently at Dawn'
Created presentation for hymn number 462 - 'I Love to Tell the Story'
Created presentation for hymn number 463 - 'How Firm a Foundation'
Created presentation for hymn number 464 - 'Our Father, Which Art in Heaven'
Created presentation for hymn number 465 - 'What a Friend We Have in Jesus'
Created presentation for hymn number 466 - 'Come and Fill Our Hearts'
Created presentation for hymn number 467 - 'Give Us Light'
Created presentation for hymn number 468 - 'In My Life'
Created presentation for hymn number 469 - 'Lord, Listen to Your Children Praying'
Created presentation for hymn number 470 - 'There Is a Longing in Our Hearts'
Created presentation for hymn number 471 - 'O Lord, Hear My Prayer'
Created presentation for hymn number 472 - 'Kum ba Yah'
Created presentation for hymn number 473 - 'Shepherd Me, O God'
Created presentation for hymn number 474 - 'As a Child Rests'
Created presentation for hymn number 475 

Created presentation for hymn number 581 - 'Glory Be to the Father'
Created presentation for hymn number 582 - 'Glory to God, Whose Goodness Shines on Me'
Created presentation for hymn number 583 - 'Glory to God'
Created presentation for hymn number 584 - 'Glory, Glory, Glory'
Created presentation for hymn number 589 - 'Alleluia'
Created presentation for hymn number 590 - 'Hallelujah'
Created presentation for hymn number 591 - 'Halle, Halle, Hallelujah!'
Created presentation for hymn number 592 - 'Holy, Holy, Holy'
Created presentation for hymn number 593 - 'Holy, Holy'
Created presentation for hymn number 594 - 'Holy, Holy, Holy, Holy'
Created presentation for hymn number 595 - 'Holy, Holy, Holy'
Created presentation for hymn number 596 - 'You Are Holy'
Created presentation for hymn number 598 - 'Amen, We Praise Your Name'
Created presentation for hymn number 599 - 'Amen'
Created presentation for hymn number 600 - 'Amen'
Created presentation for hymn number 601 - 'Amen'
Created presen

Created presentation for hymn number 709 - 'God, We Honor You'
Created presentation for hymn number 710 - 'We Lift Our Voices'
Created presentation for hymn number 711 - 'Lord of All Good'
Created presentation for hymn number 712 - 'As Those of Old Their Firstfruits Brought'
Created presentation for hymn number 713 - 'Touch the Earth Lightly'
Created presentation for hymn number 714 - 'God of the Fertile Fields'
Created presentation for hymn number 715 - 'The Earth Belongs to God Alone'
Created presentation for hymn number 716 - 'God, Whose Giving Knows No Ending'
Created presentation for hymn number 717 - 'For the Life That You Have Given'
Created presentation for hymn number 718 - 'Take Up Your Cross, the Savior Said'
Created presentation for hymn number 719 - 'Come, Labor On'
Created presentation for hymn number 720 - 'Jesus Calls Us'
Created presentation for hymn number 721 - 'Lord, You Have Come to the Lakeshore'
Created presentation for hymn number 722 - 'Lord, Speak to Me, That 

Created presentation for hymn number 824 - 'There Is a Place of Quiet Rest'
Created presentation for hymn number 825 - 'Swing Low, Sweet Chariot'
Created presentation for hymn number 826 - 'Lift High the Cross'
Created presentation for hymn number 827 - 'O Morning Star, How Fair and Bright'
Created presentation for hymn number 828 - 'More Love to Thee, O Christ'
Created presentation for hymn number 829 - 'My Faith Looks Up to Thee'
Created presentation for hymn number 830 - 'Jesus, Priceless Treasure'
Created presentation for hymn number 831 - 'I Depend upon Your Faithfulness'
Created presentation for hymn number 832 - 'Here on Jesus Christ I Will Stand'
Created presentation for hymn number 833 - 'O Love That Wilt Not Let Me Go'
Created presentation for hymn number 834 - 'Precious Lord, Take My Hand'
Created presentation for hymn number 835 - 'Just a Closer Walk with Thee'
Created presentation for hymn number 836 - 'Abide with Me'
Created presentation for hymn number 837 - 'What a Fell

In [30]:
# Create a function to share the google drive folder of slides
def find_folder_id(folder_name):
    response = drive_service.files().list(
        q=f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed=false",
        spaces='drive',
        fields='files(id, name)').execute()
    folders = response.get('files', [])
    return folders[0]['id'] if folders else None

def share_folder_with_user(folder_id, user_email):
    user_permission = {
        'type': 'user',
        'role': 'writer',  # or 'reader' if you want to restrict editing
        'emailAddress': user_email
    }
    try:
        drive_service.permissions().create(
            fileId=folder_id,
            body=user_permission,
            fields='id',
            sendNotificationEmail=True
        ).execute()
        print(f"Folder shared with {user_email}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Folder name to find and share
folder_name = 'Hymn Lyrics'

# Find the folder and share it
folder_id = find_folder_id(folder_name)
if folder_id:
    share_folder_with_user(folder_id, my_email)
else:
    print(f"Folder '{folder_name}' not found.")

Folder shared with cobleahartman@gmail.com
